In [158]:
import pandas as pd
from openpyxl import Workbook
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import *
from openpyxl.chart import *
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.chart.label import DataLabelList

import string
import os

Jika pandas dan openpyxl belum terinstall, dapat dengan mudah diinstall menggunakan command pip3 install pandas dan pip3 install openpyxl

In [72]:
os.getcwd()

'/Users/anggapradiktas/Documents/notebooks'

In [73]:
df_excel = pd.read_excel('data/supermarket_sales.xlsx')
df_excel[['Gender', 'Product line', 'Total']].head()

,Gender,Product line,Total
0,Female,Health and beauty,548.9715
1,Female,Electronic accessories,80.2200
2,Male,Home and lifestyle,340.5255
3,Male,Health and beauty,489.0480
4,Male,Sports and travel,634.3785


Pivot Table

In [74]:
report_table = df_excel.pivot_table(index='Gender',
                                  columns='Product line',
                                  values='Total',
                                  aggfunc='sum').round(0)

In [75]:
report_table

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102.0,30437.0,33171.0,18561.0,30037.0,28575.0
Male,27236.0,23868.0,22974.0,30633.0,23825.0,26548.0


In [76]:
report_table.to_excel('output/report_2021.xlsx',
                      sheet_name='Report',
                      startrow=4)

create row & column

In [86]:
wb = load_workbook('output/report_2021.xlsx')
wb.active = wb['Report']
# sheet = wb['Report']
# cell references (original spreadsheet) 
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

In [89]:
print("""
min_column: {0}
max_column: {1}
min_row: {2}
max_row: {3}
    """.format(min_column,max_column,min_row,max_row))


min_column: 1
max_column: 7
min_row: 5
max_row: 7
    


Buat excel chart melalui python

In [111]:
wb = load_workbook('output/report_2021.xlsx')
wb.active = wb['Report']
sheet = wb['Report']

# barchart
barchart = BarChart()

#locate data and categories
data = Reference(sheet,
                 min_col=min_column+1,
                 max_col=max_column,
                 min_row=min_row,
                 max_row=max_row) #including headers
categories = Reference(sheet,
                       min_col=min_column,
                       max_col=min_column,
                       min_row=min_row+1,
                       max_row=max_row) #not including headers

# adding data and categories
barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)
# props = GraphicalProperties(solidFill="000000") 
# barchart.plot_area.graphicalProperties = props

#location chart
sheet.add_chart(barchart, "B12")
barchart.title = 'Sales by Product line'
barchart.style = 2 #choose the chart style
wb.save('output/report_2021.xlsx')

Penjelasan code:
https://openpyxl.readthedocs.io/en/stable/styles.html#colours

- barchart = BarChart() memanggil barchart variable dari Barchart class
- data & categories adalah variables yang menunjukkan data informasi grafik tsb. Kita menggunakan column & row references supaya tidak perlu melakukan hardcode lagi. Header juga kita munculkan pada data. 
- add_data & set_categories digunakan untuk menambahkan data ke grafik. Didalam add_data titles_from_data=True berarti kita juga memasukkan header
- sheet.add_chart digunakan untuk menambahkan grafik ke excel file.
- barchart.title & barchart.style dapat digunakan untuk mengubah judul dan jenis grafik
- wb.save() digunakan untuk menyimpan semua perubahan

Applying Excel formulas through Python

In [33]:
import string
alphabet = list(string.ascii_uppercase)
excel_alphabet = alphabet[0:max_column] 
print(excel_alphabet)

['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [39]:
wb = load_workbook('output/report_2021.xlsx')
sheet = wb['Report']
# sum in columns B-G
for i in excel_alphabet:
    if i!='A':
        sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
        sheet[f'{i}{max_row+1}'].style = 'Currency'
# adding total label
sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
wb.save('output/report_2021.xlsx')

Formatting the report sheet

In [36]:
wb = load_workbook('output/report_2021.xlsx')
sheet = wb['Report']
sheet['A1'] = 'Sales Report'
sheet['A2'] = '2021'
sheet['A1'].font = Font('Arial', bold=True, size=20)
sheet['A2'].font = Font('Arial', bold=True, size=10)
wb.save('output/report_2021.xlsx')


Workbook 2

In [67]:
report_table_2 = df_excel[['City', 'Total']].groupby(['City']).sum().round(0)

In [98]:
# Calculating Percentage
report_table_2['percent'] = ((report_table_2['Total'] / report_table_2['Total'].sum()) * 100).round(0)

In [105]:
del report_table_2['Total']
report_table_2

,percent
City,
Mandalay,33.0
Naypyitaw,34.0
Yangon,33.0


In [106]:
report_table_2.to_excel('output/report_2021.xlsx',
                      sheet_name='City',
                      startrow=4)

In [107]:
with pd.ExcelWriter('output/report_2021.xlsx') as writer:  
    report_table.to_excel(writer, sheet_name='Report', startrow=4)
    report_table_2.to_excel(writer, sheet_name='City', startrow=4)

In [108]:
wb = load_workbook('output/report_2021.xlsx')
wb.active = wb['City']
# cell references (original spreadsheet) 
min_column_2 = wb.active.min_column
max_column_2 = wb.active.max_column
min_row_2 = wb.active.min_row
max_row_2 = wb.active.max_row

In [109]:
print("""
min_column_2: {0}
max_column_2: {1}
min_row_2: {2}
max_row_2: {3}
    """.format(min_column_2, max_column_2, min_row_2, max_row_2))


min_column_2: 1
max_column_2: 2
min_row_2: 5
max_row_2: 8
    


In [110]:
wb = load_workbook('output/report_2021.xlsx')
wb.active = wb['City']
sheet_2 = wb['City']

# piechart
piechart = PieChart3D()

#locate data and categories
data_2 = Reference(sheet_2,
                 min_col=min_column_2+1,
                 max_col=max_column_2,
                 min_row=min_row_2,
                 max_row=max_row_2) #including headers
categories_2 = Reference(sheet_2,
                       min_col=min_column_2,
                       max_col=min_column_2,
                       min_row=min_row_2+1,
                       max_row=max_row_2) #not including headers

# adding data and categories
piechart.add_data(data_2, titles_from_data=True)
piechart.set_categories(categories_2)
piechart.dataLabels = DataLabelList() 
piechart.dataLabels.showVal = True

#location chart
sheet_2.add_chart(piechart, "B12")
piechart.title = 'Sales by Region'
piechart.style = 2 #choose the chart style
wb.save('output/report_2021.xlsx')

Membuat function dari code diatas

In [181]:
# def create_excel_file(file_name, sheet_name):
#     wb = Workbook()
#     wb.create_sheet(sheet_name)
#     month_and_extension = file_name.split('_')[1]
#     excel_filename = f'output/report_{month_and_extension}'
#     wb.save(excel_filename)
#     return excel_filename

In [182]:
# create_excel_file('data/sales_2021.xlsx', 'Product line')

'output/report_2021.xlsx'

In [193]:
def automate_excel(file_name, sheet_name, index, chart_style, columns=None):
    """The file name should have the following structure: sales_month.xlsx"""
    # read excel file
    excel_file = pd.read_excel(file_name)
    # make pivot table
    report_table = excel_file.pivot_table(index=index, columns=columns, values='Total', aggfunc='sum').round(0)
    # splitting the month and extension from the file name
    month_and_extension = file_name.split('_')[1]

    # send the report table to excel file
    try:
        with pd.ExcelWriter(f'output/report_{month_and_extension}', mode='a', if_sheet_exists='replace') as writer:
            report_table.to_excel(writer, sheet_name=sheet_name, startrow=4)
    except KeyError:
        with pd.ExcelWriter(f'output/report_{month_and_extension}') as writer:
            report_table.to_excel(writer, sheet_name=sheet_name, startrow=4)
    # loading workbook and selecting sheet
    wb = load_workbook(f'output/report_{month_and_extension}')
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])
    wb.active = wb[sheet_name]
    sheet = wb[sheet_name]
    # cell references (original spreadsheet)
    min_column = wb.active.min_column
    max_column = wb.active.max_column
    min_row = wb.active.min_row
    max_row = wb.active.max_row
    # adding a chart
    chart_style = chart_style #BarChart()
    data = Reference(sheet, min_col=min_column+1, max_col=max_column, min_row=min_row, max_row=max_row) #including headers
    categories = Reference(sheet, min_col=min_column, max_col=min_column, min_row=min_row+1, max_row=max_row) #not including headers
    chart_style.add_data(data, titles_from_data=True)
    chart_style.set_categories(categories)
    sheet.add_chart(chart_style, "B12") #location chart
    chart_style.title = 'Sales by {}'.format(sheet_name)
    chart_style.style = 2 #choose the chart style
    # applying formulas
    # first create alphabet list as references for cells
    alphabet = list(string.ascii_uppercase)
    excel_alphabet = alphabet[0:max_column] #note: Python lists start on 0 -> A=0, B=1, C=2. #note2 the [a:b] takes b-a elements
    # sum in columns B-G
    for i in excel_alphabet:
        if i!='A':
            sheet[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
            sheet[f'{i}{max_row+1}'].style = 'Currency'
    sheet[f'{excel_alphabet[0]}{max_row+1}'] = 'Total'
    # getting month name
    month_name = month_and_extension.split('.')[0]
    # formatting the report
    sheet['A1'] = 'Sales Report'
    sheet['A2'] = month_name.title()
    sheet['A1'].font = Font('Arial', bold=True, size=20)
    sheet['A2'].font = Font('Arial', bold=True, size=10)
    wb.save(f'output/report_{month_and_extension}')
    

In [194]:
automate_excel('data/sales_2021.xlsx', 'Product line', 'Gender', BarChart(), 'Product line')

In [195]:
automate_excel('data/sales_2021.xlsx', 'City', 'City', PieChart())

References:
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html
- https://openpyxl.readthedocs.io/en/latest/charts/pie.html